<a href="https://colab.research.google.com/github/mhpbreugem/BBP/blob/main/Hellwig_1980.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
using LinearAlgebra, BenchmarkTools

In [31]:
# 1. Conjecture Price Function

## P = a1*Z + a2*Π

# 2. Learning

## Yi = Π + εi
## μi = b1*Y + b2*P
## σi = ...

# 3. Individual Demand

## Xi = c1*μi + c2*P = d1*Π + d2*εi

# 4. Market Clearing

## Xbar = k1*P + k2*Π

# 5. Verify Conjecture

In [32]:
function φS(A, Ψ)
    S1 = [A[1], A[2], 0.0]
    S2 = [1.0, 0.0, 1.0]
    S_full = hcat(S1, S2)
    return S_full[:, findall(Ψ .== 1)]
end

φS (generic function with 1 method)

In [33]:
function post_coef(ΣF, S)
    ΣYY = transpose(S) * ΣF * S                # 2×2
    ΣFY = ΣF * S                               # 3×2
    B = ΣFY * inv(ΣYY)                         # 3×2
    Ω = ΣF - ΣFY * inv(ΣYY) * transpose(ΣFY)   # 3×3
    return B, Ω
end

post_coef (generic function with 1 method)

In [36]:
function demand_affine(EF, SigmaF, A, Psi, rf, gamma)
    S1 = [A[1], A[2], 0.0]
    S2 = [1.0, 0.0, 1.0]
    S_full = hcat(S1, S2)
    S = S_full[:, findall(Psi .== 1)]

    B, Omega = post_coef(SigmaF, S)
    b = B[1, :]
    sigma2 = Omega[1, 1]
    expected_signals = transpose(S) * EF

    C = (b - [1 + rf, 0.0][1:length(b)]) / (gamma * sigma2)
    X0 = (EF[1] - dot(b, expected_signals)) / (gamma * sigma2)

    return X0, C
end

demand_affine (generic function with 1 method)

In [34]:
ΣF = diagm([0.25, 0.64, 1.0])
B, Ω = post_coef(ΣF, φS([1.5, 2.0], [1,1]))

([0.09966777408637875 0.1700996677740864; 0.42524916943521596 -0.1275747508305648; -0.09966777408637874 0.8299003322259136], [0.17009966777408636 -0.1275747508305648 -0.1700996677740864; -0.12757475083056477 0.09568106312292357 0.1275747508305648; -0.1700996677740864 0.1275747508305648 0.17009966777408636])

In [37]:
EF = [1.0, 0.0, 0.0]
SigmaF = diagm([0.25, 0.64, 1.0])
A = [1.5, 2.0]
Psi = [1, 1]      # use both signals: P and Y
rf = 0.01
gamma = 3.0

X0, C = demand_affine(EF, SigmaF, A, Psi, rf, gamma)

println("Demand = X0 + C[1]*P + C[2]*Y")
println("X0 = ", X0)
println("C = ", C)


Demand = X0 + C[1]*P + C[2]*Y
X0 = 1.3333333333333333
C = [-1.7839192708333336, 0.33333333333333337]


In [38]:
function phiA(EF, SigmaF, A, Psi, rf, gamma, Xbar)
    # Build signal matrix
    S1 = [A[1], A[2], 0.0]
    S2 = [1.0, 0.0, 1.0]
    S_full = hcat(S1, S2)
    S = S_full[:, findall(Psi .== 1)]

    # Posterior coefficients
    SigmaYY = transpose(S) * SigmaF * S
    SigmaFY = SigmaF * S
    B = SigmaFY * inv(SigmaYY)
    Omega = SigmaF - B * transpose(S) * SigmaF

    # Posterior mean of Pi
    b = B[1, :]
    sigma2 = Omega[1, 1]
    mu_signals = transpose(S) * EF

    C = (b - [1 + rf, 0.0][1:length(b)]) / (gamma * sigma2)
    X0 = (EF[1] - dot(b, mu_signals)) / (gamma * sigma2)

    # Compute price equation coefficients
    a1 = -1 / C[1]
    a2 = -C[2] / C[1]

    return a1, a2
end


phiA (generic function with 1 method)

In [39]:
# Prior mean and covariance
EF = [1.0, 0.0, 0.0]                        # Prior mean of F = [Pi, Z, eps]
SigmaF = diagm([0.25, 0.64, 1.0])           # Prior covariance matrix

# Signal definition
A = [1.5, 2.0]                              # Defines P = 1.5*Pi + 2.0*Z
Psi = [1, 1]                                # Use both P and Y signals

# Market parameters
rf = 0.01                                   # Risk-free rate
gamma = 3.0                                 # Risk aversion
Xbar = 1.0                                  # Target aggregate demand

# Call the function
P0, a1, a2 = phiA(EF, SigmaF, A, Psi, rf, gamma, Xbar)

# Show the results
println("Equilibrium price is: P = P0 + a1*Z + a2*Pi")
println("P0 = ", P0)
println("a1 = ", a1)
println("a2 = ", a2)


Equilibrium price is: P = P0 + a1*Z + a2*Pi
P0 = 0.18685449436151957
a1 = 0.5605634830845588
a2 = 0.18685449436151966


In [48]:
# Define phiA (as from your previous setup)
function phiA(SigmaF, A, rf, gamma)
    S1 = [A[1], A[2], 0.0]
    S2 = [1.0, 0.0, 1.0]
    S_full = hcat(S1, S2)
    S = S_full[:, findall(Psi .== 1)]

    SigmaYY = transpose(S) * SigmaF * S
    SigmaFY = SigmaF * S
    B = SigmaFY * inv(SigmaYY)
    Omega = SigmaF - B * transpose(S) * SigmaF

    b = B[1, :]
    sigma2 = Omega[1, 1]
    mu_signals = transpose(S) * EF

    C = (b - [1 + rf, 0.0][1:length(b)]) / (gamma * sigma2)
    X0 = (EF[1] - dot(b, mu_signals)) / (gamma * sigma2)

    a1 = -1 / C[1]
    a2 = -C[2] / C[1]
    return [a1, a2]
end

# Parameters
SigmaF = diagm([0.25, 0.64, 1.0])
rf = 0.01
gamma = 3.0
Xbar = 1.0

# Initial guess for A
A = [1.0, 1.0]

# Fixed-point iteration
tol = 1e-8
max_iter = 100
for iter in 1:max_iter
    A_new = phiA(SigmaF, A, rf, gamma)
    if norm(A_new - A) < tol
        println("Converged after $iter iterations.")
        break
    end
    A = A_new
end

println("Fixed-point solution: A = ", A)


Converged after 15 iterations.
Fixed-point solution: A = [-3.384202833851234e-9, -1.1280676112837466e-9]
